# step 1: prepare the dataset

In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score

def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected

def classification_binary(csvpath,startColumnName,
                          endColumnName,isdefects,
                         bugcolumnName,columncombation_j = None):

    filePath = "AlltheMatrix/"

    filePathNowusing = filePath + csvpath

    changed_matrix = load_one_version_method_matrix(filePathNowusing)
    print(f'lens: {len(changed_matrix.columns.values)}')
    matrixlen = int(len(changed_matrix))
    startindex = 0
    endindex = 0
    for i, column in enumerate(changed_matrix.columns.values):
        if(column == startColumnName):
            startindex = i
        elif (column == endColumnName):
            endindex = i
    print(f'start:{startindex},end:{endindex}') 
    target = changed_matrix[bugcolumnName].values
    data = changed_matrix.values[:,startindex+1:endindex]
    print(target.shape)
    print(f'the data shape is {data.shape}')

    #shuffleIndex
    import numpy as np
    shuffle_index = np.random.permutation(matrixlen)
    target, data = target[shuffle_index], data[shuffle_index]

    print(type(data))
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #split the train set and the test set
    X_train = data[:round(0.7*matrixlen)]
    X_test = data[round(0.7*matrixlen):]
    Y_train = target[:round(0.7*matrixlen)]
    Y_test = target[round(0.7*matrixlen):]
    print(X_train,Y_train.shape)
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@")
    #train a binary classifier
    Y_train_bugs = (Y_train > isdefects)
    Y_test_bugs = (Y_test > isdefects)

    if(columncombation_j!=None):
        print("####################################")
        X_train = get_selected_rows(columncombation_j,X_train)
        X_test = get_selected_rows(columncombation_j,X_test)
    return X_train,Y_train_bugs,X_test,Y_test_bugs


In [3]:
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                               "post")

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[16.0 155.0 13.037037037037035 ... 0.013626834381551364 0.0 0.0]
 [0.0 0.0 1.0 ... 0.0 0.017543859649122806 0.0]
 [40.0 419.0 7.633802816901407 ... 0.007802093244529021 0.0 0.0]
 ...
 [3.0 3.0 7.0 ... 0.0 0.0 0.0]
 [7.0 8.0 3.5 ... 0.014084507042253518 0.007042253521126761 0.0]
 [9.0 57.0 2.3181818181818183 ... 0.0026223776223776225 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@


## The dataset is clearly imbalanced, and I am going to use the following 5 methods to deal with it


# step 2: five ways to deal with the imbalanced dataset


In [6]:
from sklearn.metrics import accuracy_score  
from sklearn.metrics import recall_score  
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from collections import Counter

def get_all_scores(y_real, y_pre):
    return accuracy_score(y_real, y_pre), recall_score(y_real, y_pre), precision_score(y_real, y_pre), f1_score(y_real, y_pre)

def append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,dataset):
    accuracy_score_list.append(dataset[0])
    recall_score_list.append(dataset[1])
    precision_score_list.append(dataset[2])
    f1_score_list.append(dataset[3])
    
def print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list):
    print(f'The train  accuracy score is {np.mean(accuracy_score_list)}')
    print(f'The train recall score is {np.mean(recall_score_list)}')
    print(f'The train precision score is {np.mean(precision_score_list)}')
    print(f'The train f1 score is {np.mean(f1_score_list)}')

def print_all_test_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list):
    print(f'The test accuracy score is {np.mean(accuracy_score_list)}')
    print(f'The test recall score is {np.mean(recall_score_list)}')
    print(f'The test precision score is {np.mean(precision_score_list)}')
    print(f'The test f1 score is {np.mean(f1_score_list)}')

# def init_the_list(accuracy_score_list = accuracy_score_list,recall_score_list = recall_score_list,
#                   precision_score_list = precision_score_list,f1_score_list = f1_score_list,
#                  accuracy_score_list2 = accuracy_score_list2,recall_score_list2 = recall_score_list2,
#                   precision_score_list2 = precision_score_list2,f1_score_list2 = f1_score_list2):
#     print("in")
#     print(len(accuracy_score_list))
#     for i in range(len(accuracy_score_list)):
#         accuracy_score_list.pop()
#         recall_score_list.pop()
#         precision_score_list.pop()
#         f1_score_list.pop()
#     print(f'list:{len(accuracy_score_list)}')
#     for i in range(len(accuracy_score_list2)):
#         accuracy_score_list2.pop()
#         recall_score_list2.pop()
#         precision_score_list2.pop()
#         f1_score_list2.pop()
#     print(f'list2:{len(accuracy_score_list2)}')
    
    
accuracy_score_list = []
recall_score_list = []
precision_score_list = []
f1_score_list = []
accuracy_score_list2 = []
recall_score_list2 = []
precision_score_list2 = []
f1_score_list2 = []
print(f'Y_train_bugs:{sorted(Counter(Y_train_bugs).items())}')

Y_train_bugs:[(False, 6346), (True, 1069)]


# step 2.1: trying different algorithms
# K-nearest neighbors

In [14]:
from sklearn.neighbors import KNeighborsClassifier
def KNN(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    for i in range(1):
        knn = KNeighborsClassifier(n_neighbors=3,weights = 'distance')
        knn.fit(X_train,Y_train_bugs)
        Y_train_predict = knn.predict(X_train)
        Y_test_predict = knn.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    #print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
KNN(X_train,Y_train_bugs,X_test,Y_test_bugs)

The test accuracy score is 0.8414096916299559
The test recall score is 0.2817204301075269
The test precision score is 0.43521594684385384
The test f1 score is 0.34203655352480417


# Naïve Bayes,

In [15]:
from sklearn.naive_bayes import GaussianNB
def NaivceBayes(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    for i in range(10):
        NB =  GaussianNB()
        NB.fit(X_train,Y_train_bugs)
        Y_train_predict = NB.predict(X_train)
        Y_test_predict = NB.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    #print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
NaivceBayes(X_train,Y_train_bugs,X_test,Y_test_bugs)

The test accuracy score is 0.8370044052863437
The test recall score is 0.3032258064516129
The test precision score is 0.4208955223880597
The test f1 score is 0.3525


# Logistic Regression

In [19]:
from sklearn.linear_model import LogisticRegression
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"StringLiteral",0,
                               "post")
def logisticRegression(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    for i in range(5):
        LR_clf = LogisticRegression(solver='liblinear',max_iter=1000,class_weight='balanced')
        LR_clf.fit(X_train,Y_train_bugs)
        Y_train_predict = LR_clf.predict(X_train)
        Y_test_predict = LR_clf.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    #print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
logisticRegression(X_train,Y_train_bugs,X_test,Y_test_bugs)

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:80
(10593,)
the data shape is (10593, 73)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[3.0 3.0 1.5 ... 51 4 4]
 [11.0 28.0 7.0 ... 258 14 8]
 [5.0 18.0 3.333333333333333 ... 148 24 5]
 ...
 [3.0 4.0 4.333333333333333 ... 45 7 2]
 [8.0 16.0 7.375 ... 149 27 14]
 [0.0 0.0 0.0 ... 9 0 0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.76431718061674
The test recall score is 0.6420581655480985
The test precision score is 0.3276255707762557
The test f1 score is 0.43386243386243384


# Decision Tree,

In [17]:
from sklearn.tree import DecisionTreeClassifier
def DecisionTree(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    print(len(accuracy_score_list2))
    X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                                   "post")
    for i in range(10):
        DT = DecisionTreeClassifier(random_state=0,class_weight='balanced')
        DT.fit(X_train,Y_train_bugs)
        Y_train_predict = DT.predict(X_train)
        Y_test_predict = DT.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    #print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
DecisionTree(X_train,Y_train_bugs,X_test,Y_test_bugs)

0
AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[2.0 10.0 1.0 ... 0.003003003003003003 0.003003003003003003
  0.015015015015015015]
 [6.0 10.0 9.25 ... 0.0 0.0 0.0]
 [18.0 35.0 6.666666666666668 ... 0.0046801872074883 0.0015600624024961
  0.0]
 ...
 [1.0 2.0 1.6 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.8055380742605414
The test recall score is 0.3340380549682875
The test precision score is 0.34273318872017355
The test f1 score is 0.33832976445396146


# Random Forest

In [18]:
from sklearn.ensemble import RandomForestClassifier
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                                   "post")
def RandomForest(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    for i in range(5):
        RF = RandomForestClassifier(n_estimators = 200,criterion = 'entropy')
        RF.fit(X_train,Y_train_bugs)
        Y_train_predict = RF.predict(X_train)
        Y_test_predict = RF.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    #print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
RandomForest(X_train,Y_train_bugs,X_test,Y_test_bugs)

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[34.0 89.0 3.8333333333333335 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.018867924528301886 0.0 0.0]
 [30.0 1035.0 11.525547445255475 ... 0.0016653236637120602
  5.372011818426001e-05 0.0]
 ...
 [3.0 8.0 2.636363636363636 ... 0.003508771929824561 0.003508771929824561
  0.0]
 [12.0 29.0 4.0 ... 0.0 0.002202643171806168 0.0]
 [18.0 19.0 14.333333333333336 ... 0.011389521640091115 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.8663624921334172
The test recall score is 0.20144032921810698
The test precision score is 0.7279270773995761
The test f1 score is 0.31553369215812577


# SVM

In [188]:
from sklearn.svm import SVC
def SVM_clf(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    for i in range(10):
        svmclf = SVC(gamma='auto')
        svmclf.fit(X_train,Y_train_bugs)
        Y_train_predict = svmclf.predict(X_train)
        Y_test_predict = svmclf.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    #print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
SVM_clf(X_train,Y_train_bugs,X_test,Y_test_bugs)

The test accuracy score is 0.8508495909376966
The test recall score is 0.022727272727272728
The test precision score is 0.9166666666666666
The test f1 score is 0.04435483870967742


# Neural Networks

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                                   "post")
def neuralNetworks(X_train,Y_train_bugs,X_test,Y_test_bugs):
    accuracy_score_list = []
    recall_score_list = []
    precision_score_list = []
    f1_score_list = []
    accuracy_score_list2 = []
    recall_score_list2 = []
    precision_score_list2 = []
    f1_score_list2 = []
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    scaler.fit(X_test)
    X_test= scaler.transform(X_test)
    for i in range(5):
        mclf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 100))
        mclf.fit(X_train,Y_train_bugs)
        Y_train_predict = mclf.predict(X_train)
        Y_test_predict = mclf.predict(X_test)
        train_set_result = get_all_scores(Y_train_bugs,Y_train_predict)
        test_set_result = get_all_scores(Y_test_bugs,Y_test_predict)
        append_socre_to_list(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list,train_set_result)
        append_socre_to_list(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2,test_set_result)
    # print_all_train_results(accuracy_score_list,recall_score_list,precision_score_list,f1_score_list)
    print_all_test_results(accuracy_score_list2,recall_score_list2,precision_score_list2,f1_score_list2)
neuralNetworks(X_train,Y_train_bugs,X_test,Y_test_bugs)

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[33.0 89.0 5.454545454545454 ... 0.002144388849177984
  0.0014295925661186562 0.0]
 [9.0 22.0 8.2 ... 0.0024630541871921183 0.0024630541871921183 0.0]
 [1.0 2.0 1.5833333333333333 ... 0.0 0.0 0.0]
 ...
 [13.0 46.0 5.75 ... 0.00430416068866571 0.0014347202295552368 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [8.0 41.0 3.941176470588236 ... 0.0237741456166419 0.0014858841010401188
  0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.8303335431088735
The test recall score is 0.3696969696969697
The test precision score is 0.4080198016726545
The test f1 score is 0.3876870049462449


# step 2.2: oversample minority class

In [52]:
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
ros = RandomOverSampler(random_state=0)

In [53]:
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                               "post")
X_resampled, y_resampled = ros.fit_resample(X_train, Y_train_bugs)
print(f'Y_resampled: {sorted(Counter(y_resampled).items())}')

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[13.0 71.0 5.933333333333334 ... 0.01426872770511296
  0.0011890606420927464 0.0]
 [4.0 12.0 2.4166666666666665 ... 0.0 0.0 0.0]
 [5.0 9.0 6.75 ... 0.0 0.0 0.0]
 ...
 [21.0 128.0 11.882352941176471 ... 0.009993337774816787
  0.0006662225183211193 0.0]
 [1.0 3.0 1.4444444444444444 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
Y_resampled: [(False, 6323), (True, 6323)]


# step 2.2 K-nearest neighbors

In [54]:
KNN(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.751101321585903
The test recall score is 0.5021008403361344
The test precision score is 0.30138713745271123
The test f1 score is 0.37667454688731283


# step 2.2 Naïve Bayes,

In [211]:
NaivceBayes(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.8351164254247955
The test recall score is 0.2948453608247422
The test precision score is 0.44000000000000006
The test f1 score is 0.3530864197530864


# step 2.2 Logistic Regression

In [215]:
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"StringLiteral",0,
                               "post")
X_resampled, y_resampled = ros.fit_resample(X_train, Y_train_bugs)
logisticRegression(X_resampled,y_resampled,X_test,Y_test_bugs)

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:80
(10593,)
the data shape is (10593, 73)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[0.0 0.0 0.0 ... 59 24 16]
 [3.0 6.0 5.5 ... 91 8 5]
 [23.0 51.0 9.428571428571429 ... 263 41 14]
 ...
 [33.0 77.0 34.666666666666664 ... 575 42 17]
 [0.0 0.0 1.0 ... 12 1 0]
 [8.0 22.0 7.6 ... 138 19 9]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.7567652611705477
The test recall score is 0.6430205949656751
The test precision score is 0.31291759465478836
The test f1 score is 0.4209737827715355


# step 2.2 Decision Tree

In [216]:
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                               "post")
X_resampled, y_resampled = ros.fit_resample(X_train, Y_train_bugs)
DecisionTree(X_resampled,y_resampled,X_test,Y_test_bugs)

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[15.0 39.0 5.25 ... 0.003322259136212625 0.0 0.0]
 [12.0 31.0 4.5 ... 0.0036630036630036634 0.0 0.0]
 [6.0 13.0 4.0 ... 0.012072434607645876 0.0 0.0]
 ...
 [1.0 3.0 1.0 ... 0.00641025641025641 0.01282051282051282 0.0]
 [2.0 2.0 7.0 ... 0.020202020202020204 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
0
AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[19.0 23.0 10.5 ... 0.0 0.0021929824561403508 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [30.0 107.0 3.1555555555555554 ... 0.000646830530401035 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [16.0 27.0 7.75 ... 0.0054794520547945215 0.0027397260273972607 0.0]
 [2.0 6.0 2.3076923076923075 ... 0.003067484662576687 0.0 0.0]] (7415,)

# step 2.2 Random Forest

In [217]:
RandomForest(X_resampled,y_resampled,X_test,Y_test_bugs)

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [11.0 29.0 6.2857142857142865 ... 0.005747126436781609
  0.0019157088122605363 0.0]
 ...
 [3.0 5.0 5.0 ... 0.0 0.0038910505836575885 0.0]
 [8.0 57.0 5.8235294117647065 ... 0.005285412262156447 0.0 0.0]
 [8.0 15.0 11.666666666666664 ... 0.0 0.0032679738562091504 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.8742290748898679
The test recall score is 0.22069716775599124
The test precision score is 0.7071353231691951
The test f1 score is 0.33636912946972536


# step 2.2 Neural Networks

In [10]:
neuralNetworks(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.760981749528005
The test recall score is 0.514410480349345
The test precision score is 0.3058852129688798
The test f1 score is 0.38302054377503314


# step 2.2: Undersample majority class

In [26]:
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                               "post")
print(len(X_resampled[0]),len(X_test[0]))
X_resampled, y_resampled = cc.fit_resample(X_train, Y_train_bugs)
print(f'Y_resampled: {sorted(Counter(y_resampled).items())}')


AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[16.0 28.0 6.0 ... 0.002053388090349076 0.002053388090349076 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [10.0 30.0 6.25 ... 0.0 0.0 0.0]
 ...
 [33.0 133.0 9.842105263157897 ... 0.013473053892215571
  0.0007485029940119762 0.0]
 [17.0 65.0 6.9375 ... 0.009512485136741971 0.0011890606420927464 0.0]
 [4.0 7.0 3.25 ... 0.03424657534246575 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
160 160
Y_resampled: [(False, 1079), (True, 1079)]


In [27]:
len(X_resampled[0]),len(X_test[0])

(160, 160)

In [28]:
KNN(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.4845814977973568
The test recall score is 0.4703476482617587
The test precision score is 0.14294592914853946
The test f1 score is 0.21925643469971404


In [29]:
NaivceBayes(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.8470736312146003
The test recall score is 0.14519427402862986
The test precision score is 0.510791366906475
The test f1 score is 0.22611464968152864


In [30]:
logisticRegression(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.3080553807426054
The test recall score is 0.6462167689161554
The test precision score is 0.1349274124679761
The test f1 score is 0.2232426704344755


In [31]:
DecisionTree(X_resampled,y_resampled,X_test,Y_test_bugs)

0
AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[33.0 75.0 4.666666666666667 ... 0.002307692307692308 0.0 0.0]
 [3.0 9.0 4.6 ... 0.02008032128514056 0.0 0.0]
 [0.0 0.0 0.0 ... 0.04411764705882353 0.0 0.0]
 ...
 [10.0 10.0 8.0 ... 0.0 0.006211180124223602 0.0]
 [17.0 132.0 7.074074074074074 ... 0.0028517110266159697
  0.0004752851711026616 0.0]
 [46.0 375.0 15.951219512195122 ... 0.012031826120706385
  0.0001940617116242965 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.7995594713656387
The test recall score is 0.35152838427947597
The test precision score is 0.32135728542914166
The test f1 score is 0.3357664233576642


In [32]:
RandomForest(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.23826305852737567
The test recall score is 0.6736196319018405
The test precision score is 0.12715223135416193
The test f1 score is 0.21392395201778186


In [33]:
neuralNetworks(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.4502202643171806
The test recall score is 0.5222903885480572
The test precision score is 0.14426972871420235
The test f1 score is 0.22603565051422692


# step 2.3: Generate synthetic samples

In [34]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_train,Y_train_bugs,X_test,Y_test_bugs = classification_binary("modified_eclipse-metrics-files-3.0.csv",'FOUT_avg',"NORM_SuperMethodInvocation",0,
                               "post")
X_resampled, y_resampled = smote_enn.fit_resample(X_train, Y_train_bugs)
print(f'Y_resampled: {sorted(Counter(y_resampled).items())}')

AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[6.0 15.0 4.0 ... 0.0045662100456621 0.0045662100456621 0.0]
 [34.0 190.0 15.173913043478262 ... 0.008047585724282715 0.0 0.0]
 [1.0 2.0 1.25 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [2.0 2.0 1.0 ... 0.08192090395480225 0.0 0.0]
 [22.0 83.0 11.727272727272727 ... 0.05233380480905234
  0.0014144271570014145 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
Y_resampled: [(False, 3399), (True, 5975)]


In [36]:
KNN(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.5701699181875394
The test recall score is 0.7837209302325582
The test precision score is 0.2093167701863354
The test f1 score is 0.3303921568627451


In [35]:
NaivceBayes(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.8244178728760228
The test recall score is 0.5069767441860467
The test precision score is 0.3865248226950354
The test f1 score is 0.4386317907444669


In [37]:
logisticRegression(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.724669603524229
The test recall score is 0.7255813953488373
The test precision score is 0.29186155285313375
The test f1 score is 0.41627751834556365


In [38]:
DecisionTree(X_resampled,y_resampled,X_test,Y_test_bugs)

0
AlltheMatrix/modified_eclipse-metrics-files-3.0.csv
lens: 203
start:6,end:167
(10593,)
the data shape is (10593, 160)
<class 'numpy.ndarray'>
@@@@@@@@@@@@@@@@@@@@@@@@@@
[[12.0 29.0 10.666666666666666 ... 0.021645021645021644
  0.004329004329004329 0.0]
 [25.0 134.0 14.818181818181818 ... 0.008888888888888889
  0.0007407407407407407 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 ...
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [0.0 0.0 0.0 ... 0.0 0.0 0.0]
 [10.0 41.0 3.7647058823529416 ... 0.005641748942172072 0.0 0.0]] (7415,)
@@@@@@@@@@@@@@@@@@@@@@@@@@
The test accuracy score is 0.7920075519194462
The test recall score is 0.31941544885177453
The test precision score is 0.3135245901639344
The test f1 score is 0.3164426059979317


In [39]:
RandomForest(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.7675582127123978
The test recall score is 0.6695348837209303
The test precision score is 0.3255134082584211
The test f1 score is 0.4380416799368483


In [40]:
neuralNetworks(X_resampled,y_resampled,X_test,Y_test_bugs)

The test accuracy score is 0.6634361233480176
The test recall score is 0.6897674418604651
The test precision score is 0.2361646247041859
The test f1 score is 0.3428917117554722
